In [11]:
## Laufzeit 1 min 30 sec

import pandas as pd

df = pd.read_csv('/Users/davidsanders/Documents/GitHub/masterarbeit-flex-kpis/data/epex/lastgang_lkw/lastgang_lkw_cl_2_quote_80-80-80_netz_100_pow_100-100-100_pause_45-540_B_2_Bidirektional.csv', sep=';', decimal=',')

In [12]:
df

,LKW_ID,Datum,Woche,Tag,Zeit,Ladetyp,Ladestrategie,Ladezeit,DayAhead,Kosten_DayAhead,Intraday,Kosten_Intraday,Leistung,SOC,Max_Leistung,Pplus,Pminus,z
0,1,2024-01-01 00:15:00,1,1,15,NCS,DayAhead,0,0.006877,-0.0573,30.360,-253.0000,-100.0,0.201897,100.0,0.0,100.0,0.0
1,1,2024-01-01 00:20:00,1,1,20,NCS,DayAhead,5,0.005965,-0.0497,21.892,-182.4333,-100.0,0.188008,100.0,0.0,100.0,0.0
2,1,2024-01-01 00:25:00,1,1,25,NCS,DayAhead,10,0.005118,-0.0427,6.933,-57.7750,-100.0,0.174119,100.0,0.0,100.0,0.0
3,1,2024-01-01 00:30:00,1,1,30,NCS,DayAhead,15,0.004336,-0.0361,-2.890,24.0833,-100.0,0.160230,100.0,0.0,100.0,0.0
4,1,2024-01-01 00:35:00,1,1,35,NCS,DayAhead,20,0.003618,-0.0301,-5.640,47.0000,-100.0,0.146341,100.0,0.0,100.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17415603,115924,2024-12-31 00:25:00,53,1,25,MCS,T_min,25,5.003313,0.0000,185.341,0.0000,0.0,0.572500,1000.0,0.0,0.0,1.0
17415604,115924,2024-12-31 00:30:00,53,1,30,MCS,T_min,30,4.902124,0.0000,184.270,0.0000,0.0,0.572500,1000.0,0.0,0.0,1.0
17415605,115924,2024-12-31 00:35:00,53,1,35,MCS,T_min,35,4.796345,0.0000,180.083,0.0000,0.0,0.572500,1000.0,0.0,0.0,1.0
17415606,115924,2024-12-31 00:40:00,53,1,40,MCS,T_min,40,4.712069,0.0000,172.100,0.0000,0.0,0.572500,1000.0,0.0,0.0,1.0


In [13]:
max_kosten_intraday_id = df.loc[df['Kosten_Intraday'].idxmin(), 'LKW_ID']
max_kosten_intraday_id

75804

In [ ]:
kosten_intraday = {
    'Intraday': {'NCS': 0, 'HPC': 0, 'MCS': 0},
    'T_min':     {'NCS': 0, 'HPC': 0, 'MCS': 0},
    'Konstant': {'NCS': 0, 'HPC': 0, 'MCS': 0}
}
energie_intraday = {
    'Intraday': {'NCS': 0, 'HPC': 0, 'MCS': 0},
    'T_min':     {'NCS': 0, 'HPC': 0, 'MCS': 0},
    'Konstant': {'NCS': 0, 'HPC': 0, 'MCS': 0}
}    
    
def filter_df(ladetyp, strategie):
    return df[(df['Ladetyp'] == ladetyp) & (df['Ladestrategie'] == strategie)].copy()

df['Energie'] = df['Leistung'] * (5/60)  # Leistung in kW * Zeit in Stunden = Energie in kWh

for ladetyp in ['NCS', 'HPC', 'MCS']:
    for strategie in ['Intraday', 'T_min', 'Konstant']:
        df_filtered = filter_df(ladetyp, strategie)
        
        if not df_filtered.empty:
            # Kosten und Energie pro LKW berechnen
            kosten_pro_lkw = df_filtered.groupby('LKW_ID')['Kosten_Intraday'].sum() / 10  # Division wie im Original
            energie_pro_lkw = df_filtered.groupby('LKW_ID')['Energie'].sum()
            
            # Nur LKWs mit tatsächlich geladener Energie berücksichtigen
            valid_lkws = energie_pro_lkw[energie_pro_lkw > 0].index
            kosten_pro_lkw = kosten_pro_lkw.loc[valid_lkws]
            energie_pro_lkw = energie_pro_lkw.loc[valid_lkws]
            
            # if strategie == 'Intraday' and ladetyp == 'HPC':
            #     # print(kosten_pro_lkw)
            #     # print(energie_pro_lkw)
            #     print(kosten_pro_lkw.mean())
            #     print(energie_pro_lkw.mean())
                
            # Kosten pro kWh für jeden LKW berechnen
            if not energie_pro_lkw.empty:
                kosten_pro_kwh = kosten_pro_lkw / energie_pro_lkw
                
                print(kosten_pro_kwh.min())
                kosten_intraday[strategie][ladetyp] = kosten_pro_lkw.mean() / energie_pro_lkw.mean()  
                energie_intraday[strategie][ladetyp] = energie_pro_lkw.mean()
                
                
print(kosten_intraday)

-193.3089345444373
-2.6133719445156065
-1.1928898012663185
-1.2647125285697164e+16
-185.28467702566726
-189.46952216414388
-235.45556658934308
-210.75191785346186
-205.86670000618662
{'Intraday': {'NCS': 5.565264526413364, 'HPC': 7.831434199169795, 'MCS': 7.666757467723075}, 'T_min': {'NCS': 10.018161460262766, 'HPC': 8.848773166972233, 'MCS': 10.251016937226137}, 'Konstant': {'NCS': 9.23383161668108, 'HPC': 8.82440988679655, 'MCS': 10.286955269819986}}


In [15]:
print(kosten_intraday)

{'Intraday': {'NCS': 5.361090617847485, 'HPC': -363370931919.0225, 'MCS': 7.5014736497736045}, 'T_min': {'NCS': 10.140975728706122, 'HPC': 8.785983109397717, 'MCS': 10.126095396202652}, 'Konstant': {'NCS': 9.27937333546843, 'HPC': 8.743225075793315, 'MCS': 10.168270104417074}}


In [17]:
df[df['Ladetyp'] == 'HPC']

,LKW_ID,Datum,Woche,Tag,Zeit,Ladetyp,Ladestrategie,Ladezeit,DayAhead,Kosten_DayAhead,Intraday,Kosten_Intraday,Leistung,SOC,Max_Leistung,Pplus,Pminus,z,Energie
3924,10,2024-01-01 06:45:00,1,1,405,HPC,DayAhead,0,-0.002826,-0.0824,13.080,381.5000,350.000000,0.429351,350.0,350.000000,0.0,1.0,29.166667
3925,10,2024-01-01 06:50:00,1,1,410,HPC,DayAhead,5,-0.002524,-0.0736,9.411,274.4875,350.000000,0.459733,350.0,350.000000,0.0,1.0,29.166667
3926,10,2024-01-01 06:55:00,1,1,415,HPC,DayAhead,10,-0.002249,-0.0656,3.015,87.9375,350.000000,0.490115,350.0,350.000000,0.0,1.0,29.166667
3927,10,2024-01-01 07:00:00,1,1,420,HPC,DayAhead,15,-0.002000,-0.0583,2.680,78.1667,350.000000,0.520497,350.0,350.000000,0.0,1.0,29.166667
3928,10,2024-01-01 07:05:00,1,1,425,HPC,DayAhead,20,-0.001777,-0.0518,9.872,287.9333,350.000000,0.550879,350.0,350.000000,0.0,1.0,29.166667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17396687,115840,2024-12-30 20:50:00,53,1,1250,HPC,T_min,25,6.825483,199.0766,135.767,3959.8708,350.000000,0.494003,350.0,350.000000,0.0,1.0,29.166667
17396688,115840,2024-12-30 20:55:00,53,1,1255,HPC,T_min,30,6.807845,198.5621,163.232,4760.9333,350.000000,0.524385,350.0,350.000000,0.0,1.0,29.166667
17396689,115840,2024-12-30 21:00:00,53,1,1260,HPC,T_min,35,6.802000,115.7978,185.880,3164.4365,204.288996,0.554767,350.0,204.288996,0.0,1.0,17.024083
17396690,115840,2024-12-30 21:05:00,53,1,1265,HPC,T_min,40,6.807950,0.0000,201.093,0.0000,0.000000,0.572500,350.0,0.000000,0.0,1.0,0.000000
